### Let's prepare the data for fine-tuning !

In [4]:
import sqlite3
import pandas as pd
import json
from data_preparation.base_prep import *
from data_preparation.cm_prep import structurally_causal_mask_elements_and_save


In [24]:
token_max_count = 4000
producing_datasets_structurally_non_contiguous(max_token_count=token_max_count,percentage= 0.2)
producing_datasets_at_random_contiguous(max_token_count=token_max_count,percentage= 0.2)
producing_datasets_structurally_contiguous(max_token_count=token_max_count,percentage= 0.2)

Total nodes before removal: 39
Number of nodes to remove: 7
nodes in removed_nodes: 7
nodes in remaining_nodes: 32
nodes without links: 2
nodes in removed_nodes after adding unlinked nodes: 9
Processed file: __M6nYHZ-EeqQk6mCratp8w.json
Percentage of nodes removed in __M6nYHZ-EeqQk6mCratp8w.json: 23.08%
Total nodes before removal: 47
Number of nodes to remove: 9
nodes in removed_nodes: 9
nodes in remaining_nodes: 38
nodes without links: 3
nodes in removed_nodes after adding unlinked nodes: 12
Processed file: 6c114e10-24cc-4282-8ab0-0b379e2145e9.json
Percentage of nodes removed in 6c114e10-24cc-4282-8ab0-0b379e2145e9.json: 25.53%
Total nodes before removal: 32
Number of nodes to remove: 6
nodes in removed_nodes: 6
nodes in remaining_nodes: 26
nodes without links: 1
nodes in removed_nodes after adding unlinked nodes: 7
Processed file: _9vFQwKoTEeeEQN1ZyOr-_g.json
Percentage of nodes removed in _9vFQwKoTEeeEQN1ZyOr-_g.json: 21.88%
Total nodes before removal: 40
Number of nodes to remove: 

In [ ]:
input_folder = 'json_dataset/json2000'
input_output_folder = 'fine_tuning_raw_datasets/processed_input_mask'
output_folder = 'fine_tuning_raw_datasets/processed_output_mask'

# Define the percentage of elements to remove
percentage = 0.15  # 20%

masks = structurally_causal_mask_elements_and_save(
    input_folder=input_folder,
    input_output_folder=input_output_folder,
    output_folder=output_folder,
    percentage=percentage
)

In [34]:
from util_folder.json_extraction_util import read_file

dataset_type = 'structural_removal_contiguous'
#dataset_type = 'structural_removal_non_contiguous'
#dataset_type = 'random_removal'
token_count_for_dataset = 4000

def read_json_files_from_folder(folder_path):
    data = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        a = read_file(file_path)
        data.append(a)
    return data

data2000_input = read_json_files_from_folder(f"fine_tuning_raw_datasets/{dataset_type}/processed_{token_count_for_dataset}/processed_input{token_count_for_dataset}")
data2000_output = read_json_files_from_folder(f"fine_tuning_raw_datasets/{dataset_type}/processed_{token_count_for_dataset}/processed_output{token_count_for_dataset}")

# Convert the JSON data to DataFrames
df = pd.DataFrame(data2000_input, columns=['input'])
df['output'] = data2000_output
df

,input,output
0,"{""directed"":true,""nodes"":[{""isDerived"":false,""...","{""nodes"":[{""viewpoint"":null,""visibility"":""PUBL..."
1,"{""directed"":true,""nodes"":[{""viewpoint"":null,""v...","{""nodes"":[{""isDerived"":false,""visibility"":""PUB..."
2,"{""directed"":true,""nodes"":[{""viewpoint"":null,""v...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."
3,"{""directed"":true,""nodes"":[{""viewpoint"":null,""v...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."
4,"{""directed"":true,""nodes"":[{""viewpoint"":null,""v...","{""nodes"":[{""isStatic"":false,""isDerived"":false,..."
...,...,...
721,"{""directed"":true,""nodes"":[{""viewpoint"":null,""v...","{""nodes"":[{""isStatic"":false,""isDerived"":false,..."
722,"{""directed"":true,""nodes"":[{""viewpoint"":null,""v...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."
723,"{""directed"":true,""nodes"":[{""viewpoint"":null,""v...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."
724,"{""directed"":true,""nodes"":[{""viewpoint"":null,""v...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."


In [35]:
#take the first row of column output
print(df['output'][0])

{"nodes":[{"viewpoint":null,"visibility":"PUBLIC_LITERAL","qualifiedName":"ASSIGNMENT","name":"ASSIGNMENT","id":0,"URI":null,"eClass":"Model"},{"visibility":"PUBLIC_LITERAL","id":1,"eClass":"PackageImport"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ASSIGNMENT::Use Case Diagram of Student Attendance System","name":"Use Case Diagram of Student Attendance System","isIndirectlyInstantiated":true,"id":2,"isActive":false,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"Component"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ASSIGNMENT::Actor","name":"Actor","id":3,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"Actor"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ASSIGNMENT::UseCase10","name":"UseCase10","id":4,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"UseCase"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ASSIGNMENT::Actor2","name":"Actor2","id":5,"isLeaf":false,"isAbstract":false,"isFinalSpec

In [37]:
def create_folder_and_dataset(train_df,validation_df,test_df,folder_name):
    # Create the folder
    os.makedirs(folder_name, exist_ok=True)
    # Save the exploring_datasets
    create_dataset(f"{folder_name}/train.jsonl", train_df)
    create_dataset(f"{folder_name}/validation.jsonl", validation_df)
    create_dataset(f"{folder_name}/test.jsonl", test_df)



def create_dataset(jsonl_file_path, dataset_df):
    with open(jsonl_file_path, 'w') as file:
        for _, row in dataset_df.iterrows():
            # Construct the json object for each line
            json_line = {"input": row['input'], "output": row['output']}
            # Convert the dictionary to a JSON string and write it to the file
            file.write(json.dumps(json_line) + '\n')

In [38]:
from sklearn.model_selection import train_test_split

def split_dataframe(df, train_size=0.8, validation_size=0.2, test_size=0.1, random_state=None):

    # First split to separate out the training set
    train_df, temp_df = train_test_split(df, train_size=train_size, random_state=random_state)

    # Adjust the proportion sizes for validation and test splits
    proportion = validation_size / (validation_size + test_size)

    # Second split to separate out the validation and test sets
    validation_df, test_df = train_test_split(temp_df, train_size=proportion, random_state=random_state)

    return train_df, validation_df, test_df

train_df, validation_df, test_df = split_dataframe(df)
# Save the exploring_datasets
create_folder_and_dataset(train_df,validation_df,test_df,f"fine_tuning_datasets/{dataset_type}/processed_" + str(token_count_for_dataset))  

In [44]:
import shutil
import os

def zip_folder(output_path, folder_path):
    shutil.make_archive(output_path, 'zip', folder_path)

In [45]:
list_of_folders = [
    'fine_tuning_datasets/random_removal/processed_2000',
    'fine_tuning_datasets/structural_removal_contiguous/processed_2000',
    'fine_tuning_datasets/structural_removal_non_contiguous/processed_2000',
    'fine_tuning_datasets/random_removal/processed_4000',
    'fine_tuning_datasets/structural_removal_contiguous/processed_4000',
    'fine_tuning_datasets/structural_removal_non_contiguous/processed_4000'
]
for folder in list_of_folders:
    zip_folder( folder, folder)